# Multiblock pipeline

In [ ]:
!python ../../retrieval/generate_description_embeddings.py \
  --data_dir ../../data/ \
  --out_dir embeddings/ \
  --model sentence-transformers/all-MiniLM-L6-v2 \
  --batch_size 32 \
  --pooling mean


In [3]:
!python ../../retrieval/retrieval_by_bloc/generate_multiblock_embeddings.py \
  --model sentence-transformers/all-MiniLM-L6-v2 \
  --data_dir ../../data/ \
  --out_dir embeddings/ \
  --pooling mean \
  --batch_size 32


Using device: cpu

=== Processing split: train ===

Encoding block: intro
100%|█████████████████████████████████████████| 969/969 [10:31<00:00,  1.54it/s]
Saved → embeddings/train_intro_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv

Encoding block: structure
100%|█████████████████████████████████████████| 969/969 [09:52<00:00,  1.63it/s]
Saved → embeddings/train_structure_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv

Encoding block: role
100%|█████████████████████████████████████████| 969/969 [09:18<00:00,  1.73it/s]
Saved → embeddings/train_role_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv

Encoding block: taxonomy
100%|█████████████████████████████████████████| 969/969 [08:01<00:00,  2.01it/s]
Saved → embeddings/train_taxonomy_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv

=== Processing split: validation ===

Encoding block: intro
100%|███████████████████████████████████████████| 32/32 [00:08<00:00,  3.75it/s]
Saved → embeddings/validation_intro_emb

Train GPS

In [6]:
!python ../../retrieval/gps_encoder/train.py \
--data_dir ../../data/ \
--train_emb ./embeddings/train_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv \
--val_emb ./embeddings/validation_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv \
--out_ckpt model.pt \
--epochs 2 


/Users/ilan/Code/MVA/Altegrad-Project-MVA/retrieval/gps_encoder
Device: cpu
Loading graphs from: ../../data/train_graphs.pkl
Loaded 31008 graphs
Loading graphs from: ../../data/validation_graphs.pkl
Loaded 1000 graphs
Trainable params: 3,642,756
Epoch 001 | loss=1.3352 | MRR=0.4307 | R@1=0.2730 | R@5=0.6140
  → New best MRR: 0.4307 | saved model.pt
Epoch 002 | loss=0.8691 | MRR=0.5190 | R@1=0.3680 | R@5=0.7060
  → New best MRR: 0.5190 | saved model.pt

Training finished. Best MRR = 0.5190


Multiblock Retrieval 

In [10]:
!python ../../retrieval/retrieval_by_bloc/retrieval_multiblock.py \
  --encoder gps \
  --model_path model.pt \
  --train_graphs ../../data/train_graphs.pkl \
  --test_graphs ../../data/test_graphs.pkl \
  --train_intro_emb embeddings/train_intro_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv \
  --train_structure_emb embeddings/train_structure_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv \
  --train_role_emb embeddings/train_role_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv \
  --train_taxonomy_emb embeddings/train_taxonomy_embeddings_sentence-transformers_all-MiniLM-L6-v2.csv \
  --output_csv submission.csv \
  --top_k 5


Device: cpu
Loading model from model.pt
Loading graphs from: ../../data/test_graphs.pkl
Loaded 1000 graphs
Retrieving (multi-block):   0%|                        | 0/1000 [00:00<?, ?it/s]
Test 0
  28246: 1.064
  10876: 0.836
  10502: 0.778
  21343: 0.772
  14920: 0.763
→ Selected:
The molecule is a D-galactosyl-N-acylsphingosine in which the glycosidic bond has beta-configuration and the ceramide N- ...
Retrieving (multi-block):   0%|                | 1/1000 [00:00<02:14,  7.45it/s]
Test 1
  13749: 0.960
  2584: 0.766
  21443: 0.625
  17893: 0.596
  14096: 0.553
→ Selected:
The molecule is a magnesium salt comprising of two chlorine atoms bound to a magnesium atom. It is a magnesium salt, an  ...

Test 2
  11290: 1.254
  2463: 1.252
  4757: 0.992
  15200: 0.843
  2253: 0.782
→ Selected:
The molecule is a steroid glucosiduronic acid that is 4-hydroxy-17beta-estradiol having a single beta-D-glucuronic acid  ...
Retrieving (multi-block): 100%|████████████| 1000/1000 [00:06<00:00, 145.07it